<a href="https://colab.research.google.com/github/mamekin05108/signatecup2024summer/blob/main/lightcat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
!pip install catboost

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold

from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool
from sklearn.isotonic import IsotonicRegression


# warningsを非表示にする
warnings.filterwarnings("ignore")

In [28]:
df_train = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_test.csv")
ss = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/data/sample_submit.csv", header=None)

In [29]:
df_train.head()

,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,...,Passport,PitchSatisfactionScore,Designation,MonthlyIncome,ProdTaken,MaritalStatus,OwnCar,NumberOfChildren,Agebin,Incomebin
0,0,50.0,Self Enquiry,2,15.0,Large Business,MALE,1.0,4.0,BASIC,...,1,4,Executive,253905.0,1.0,未婚,なし,0.0,41-50,<300000
1,1,56.0,Company Invited,1,14.0,Salaried,MALE,1.0,4.0,STANDARD,...,1,4,Senior Manager,404475.0,0.0,離婚,あり,0.0,51-65,<450000
2,2,NaN,Self Enquiry,1,10.0,Large Business,FEMALE,1.0,3.0,BASIC,...,0,4,Executive,278145.0,1.0,結婚,なし,0.0,NaN,<300000
3,3,37.0,Self Enquiry,2,18.0,Small Business,FEMALE,1.0,3.0,STANDARD,...,0,5,Senior Manager,326805.0,0.0,離婚,あり,0.0,31-40,<350000
4,4,48.0,Company Invited,3,17.0,Small Business,FEMALE,1.0,3.0,BASIC,...,0,4,Executive,258435.0,1.0,独身,あり,0.0,41-50,<300000


In [30]:
print(df_train.shape)
print(df_train)

(3489, 22)
        id   Age    TypeofContact  CityTier  DurationOfPitch      Occupation  \
0        0  50.0     Self Enquiry         2             15.0  Large Business   
1        1  56.0  Company Invited         1             14.0        Salaried   
2        2   NaN     Self Enquiry         1             10.0  Large Business   
3        3  37.0     Self Enquiry         2             18.0  Small Business   
4        4  48.0  Company Invited         3             17.0  Small Business   
...    ...   ...              ...       ...              ...             ...   
3484  3484  40.0     Self Enquiry         2             26.0        Salaried   
3485  3485  40.0     Self Enquiry         1              9.0  Large Business   
3486  3486  31.0     Self Enquiry         1             14.0  Small Business   
3487  3487  56.0  Company Invited         2             15.0        Salaried   
3488  3488  42.0     Self Enquiry         1              9.0  Small Business   

      Gender  NumberOfPerson

In [31]:
rows = []
for col in df_train.columns:
    rows.append([col, df_train[col].dtype, df_train[col].isnull().sum(), len(df_train[col].unique())])
pd.DataFrame(rows, columns=["列名", "列の型", "NaNである行の数", "値の種類"])

,列名,列の型,NaNである行の数,値の種類
0,id,int64,0,3489
1,Age,float64,100,45
2,TypeofContact,object,6,3
3,CityTier,int64,0,3
4,DurationOfPitch,float64,121,34
5,Occupation,object,0,3
6,Gender,object,0,2
7,NumberOfPersonVisiting,float64,0,4
8,NumberOfFollowups,float64,33,7
9,ProductPitched,object,0,5


In [32]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
id,3489.0,1744.000000,1007.331872,0.0,872.0,1744.0,2616.0,3488.0
Age,3389.0,39.022721,9.779403,18.0,31.0,38.0,47.0,61.0
CityTier,3489.0,1.697048,0.671500,1.0,1.0,2.0,2.0,3.0
DurationOfPitch,3368.0,14.344715,6.954363,4.0,9.0,13.0,16.0,36.0
NumberOfPersonVisiting,3489.0,2.394669,0.726612,1.0,2.0,2.0,3.0,4.0
NumberOfFollowups,3456.0,3.668981,0.889123,1.0,3.0,4.0,4.0,6.0
PreferredPropertyStar,3489.0,3.441101,0.599633,3.0,3.0,3.0,4.0,5.0
NumberOfTrips,3467.0,3.194693,1.732849,1.0,2.0,3.0,5.0,8.0
Passport,3489.0,0.102608,0.303490,0.0,0.0,0.0,0.0,1.0
PitchSatisfactionScore,3489.0,2.513901,1.115412,1.0,2.0,2.0,3.0,5.0


In [33]:
# Config cell
target = "ProdTaken"
path_in_folder = "./"

cols_category = [
    "Gender",
    "ProductPitched",
    "Designation",
    "MaritalStatus",
    "OwnCar",
    "Occupation",
    "TypeofContact",
    "Agebin",
    "Incomebin"
]

In [34]:
def preprocess(df, replace_dict=None, ce_dict=None):
    # カテゴリカル変数の設定
    df[cols_category] = df[cols_category].fillna(-1)
    if replace_dict is None:
        # countencode, labelencode
        # ce_dict: 列名を入れるとそのカテゴリのデータがどのくらいあるかを返してくれます
        # replace_dict: 列名を入れるとlabelencodeのための数字を返してくれます
        ce_dict = {}
        replace_dict = {}
        for col in cols_category:
            replace_dict[col] = {}
            vc = df[col].value_counts()
            ce_dict[col] = vc
            replace_dict_in_dict = {}
            for i, k in enumerate(vc.keys()):
                replace_dict_in_dict[k] = i
            replace_dict[col] = replace_dict_in_dict
            df[f"{col}_CountEncode"] = df[col].replace(vc).astype(int)
            df[col] = df[col].replace(replace_dict_in_dict).astype(int)
        return df, replace_dict, ce_dict

    # test
    else:
        for col in cols_category:
            # カウントエンコード
            test_vals_uniq = df[col].unique()
            ce_dict_in_dict = ce_dict[col]
            for test_val in test_vals_uniq:
                if test_val not in ce_dict_in_dict.keys():
                    ce_dict_in_dict[test_val] = -1
            df[f"{col}_CountEncode"] = df[col].replace(ce_dict_in_dict).astype(int)

            # LabelEncode
            test_vals_uniq = df[col].unique()
            replace_dict_in_dict = replace_dict[col]
            for test_val in test_vals_uniq:
                if test_val not in replace_dict_in_dict.keys():
                    replace_dict_in_dict[test_val] = -1
            df[col] = df[col].replace(replace_dict_in_dict).astype(int)
        return df

In [35]:
df_train, replace_dict, ce_dict = preprocess(df_train)
df_test = preprocess(df_test, replace_dict=replace_dict, ce_dict=ce_dict)

In [36]:
df_train.head()


,id,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,...,Incomebin,Gender_CountEncode,ProductPitched_CountEncode,Designation_CountEncode,MaritalStatus_CountEncode,OwnCar_CountEncode,Occupation_CountEncode,TypeofContact_CountEncode,Agebin_CountEncode,Incomebin_CountEncode
0,0,50.0,0,2,15.0,2,0,1.0,4.0,0,...,1,2064,1157,1150,581,1785,361,2194,843,832
1,1,56.0,1,1,14.0,1,0,1.0,4.0,2,...,3,2064,841,854,878,1704,1401,1289,527,318
2,2,NaN,0,1,10.0,2,1,1.0,3.0,0,...,1,1425,1157,1150,1384,1785,361,2194,100,832
3,3,37.0,0,2,18.0,0,1,1.0,3.0,2,...,0,1425,841,854,878,1704,1727,2194,1126,1003
4,4,48.0,1,3,17.0,0,1,1.0,3.0,0,...,1,1425,1157,1150,646,1704,1727,1289,843,832


In [37]:
rows = []
for col in df_train.columns:
    rows.append([col, df_train[col].dtype, df_train[col].isnull().sum(), len(df_train[col].unique())])
pd.DataFrame(rows, columns=["列名", "列の型", "NaNである行の数", "値の種類"])

,列名,列の型,NaNである行の数,値の種類
0,id,int64,0,3489
1,Age,float64,100,45
2,TypeofContact,int64,0,3
3,CityTier,int64,0,3
4,DurationOfPitch,float64,121,34
5,Occupation,int64,0,3
6,Gender,int64,0,2
7,NumberOfPersonVisiting,float64,0,4
8,NumberOfFollowups,float64,33,7
9,ProductPitched,int64,0,5


In [ ]:
#df_train.drop(["id"], axis=1, inplace=True)
#df_test.drop(["id"], axis=1, inplace=True)

In [38]:
train_y = df_train[target]
train_x = df_train.drop(target, axis=1)

In [ ]:
#df_train.to_csv("/content/drive/MyDrive/Signate/2024summer/dft_train.csv", index=False)
#df_test.to_csv("/content/drive/MyDrive/Signate/2024summer/dft_test.csv", index=False)

# モデル


In [39]:
params_lgb = {
    "n_estimators": 3000,
    "learning_rate": 0.01,
    "colsample_bytree": 0.8,
    "subsample_freq": 1,
    "subsample": 0.8,
    "random_seed": 0,
}

In [40]:
params_cat = {
    "iterations": 3000,  # n_estimatorsに相当
    "learning_rate": 0.01,
    "colsample_bylevel": 0.8,  # colsample_bytreeに相当
    "random_seed": 0,
    "verbose": 100,  # 学習の進捗を100回毎に表示
    "use_best_model": True,  # 早期停止のための設定
}

In [41]:
models = {
    'LightGBM': lgb.LGBMClassifier(**params_lgb),
    'CatBoost': CatBoostClassifier(**params_cat, early_stopping_rounds=100)
}

In [42]:
# f1スコアが最も高くなる点を見つける
from sklearn import metrics
def decide_cutoff(val_y, preds_y_proba):
    mean_f1_list = []
    fpr, tpr, thresholds = metrics.roc_curve(val_y, preds_y_proba)
    for threshold in thresholds:
        preds_y = [1 if prob > threshold else 0 for prob in preds_y_proba]
        #preds_y=preds_y_proba
        mean_f1_list.append(f1_score(val_y, preds_y, average='macro'))
    return np.max(mean_f1_list), thresholds[np.argmax(mean_f1_list)]

In [50]:
list_metrics_auc = []
list_metrics_f1 = []
list_cutoff = []
list_models = []

# 第1層モデルの予測を格納するためのリスト
meta_features = np.zeros((len(train_x), len(models)))

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
for fold, (trn_idx, val_idx) in enumerate(cv.split(train_x, train_y), start=1):
    trn_x = train_x.iloc[trn_idx, :]
    trn_y = train_y.iloc[trn_idx]
    val_x = train_x.iloc[val_idx, :]
    val_y = train_y.iloc[val_idx]

    for i, (model_name, model) in enumerate(models.items()):
        if model_name.startswith('LightGBM'):
            # LightGBMモデルの訓練
            model.fit(
                trn_x, trn_y,
                eval_set=[(val_x, val_y)],
                callbacks=[lgb.early_stopping(100, verbose=True)],
                categorical_feature=cols_category,
            )
        else:
            # CatBoostモデルの訓練
            train_pool = Pool(data=trn_x, label=trn_y, cat_features=cols_category)
            val_pool = Pool(data=val_x, label=val_y, cat_features=cols_category)
            model.fit(train_pool, eval_set=val_pool)

        list_models.append(model)
        preds_y_proba = model.predict_proba(val_x)[:, 1]
        meta_features[val_idx, i] = preds_y_proba

        auc = roc_auc_score(val_y, preds_y_proba)
        f1, threshold = decide_cutoff(val_y, preds_y_proba)

        list_metrics_auc.append(auc)
        list_metrics_f1.append(f1)
        list_cutoff.append(threshold)


        print(f"Fold: {fold}, Model: {model_name}, AUC: {auc:.4f}, F1 Score: {f1:.4f}, Threshold: {threshold:.4f}")


[LightGBM] [Info] Number of positive: 397, number of negative: 2394
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000610 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 728
[LightGBM] [Info] Number of data points in the train set: 2791, number of used features: 30
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.142243 -> initscore=-1.796785
[LightGBM] [Info] Start training from score -1.796785
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[321]	valid_0's binary_logloss: 0.295597
Fold: 1, Model: LightGBM, AUC: 0.8520, F1 Score: 0.7380, Threshold: 0.3369
0:	learn: 0.6839455	test: 0.6839763	best: 0.6839763 (0)	total: 10.3ms	remaining: 31s
100:	learn: 0.3480642	test: 0.3527569	best: 0.3527569 (100)	total: 1.49s	remaining: 42.8s
200:	learn: 0.2996886	test: 0.3097086	best: 0.3097086 (200)

dict_items([('LightGBM', LGBMClassifier(colsample_bytree=0.8, learning_rate=0.01, n_estimators=3000,
               random_seed=0, subsample=0.8, subsample_freq=1)), ('CatBoost', <catboost.core.CatBoostClassifier object at 0x7941ad732500>)])

In [51]:
# メタモデル用の特徴量とラベル
meta_x = meta_features
meta_y = train_y

In [52]:
meta_x

array([[0.79354267, 0.71738143],
       [0.17569248, 0.14066005],
       [0.21706225, 0.2325564 ],
       ...,
       [0.0488921 , 0.04645743],
       [0.2457984 , 0.34684678],
       [0.08048549, 0.10449203]])

In [56]:
# ロジスティック回帰のメタモデルの訓練
from sklearn.linear_model import LogisticRegression
meta_model = LogisticRegression()
meta_model.fit(meta_x, meta_y)

# メタモデルの予測
meta_predictions = meta_model.predict_proba(meta_x)[:, 1]

# 評価指標の計算
auc = roc_auc_score(meta_y, meta_predictions)
f1 = f1_score(meta_y, (meta_predictions >= 0.5).astype(int))

print(f'Meta Model AUC: {auc:.4f}')
print(f'Meta Model F1 Score: {f1:.4f}')

Meta Model AUC: 0.8368
Meta Model F1 Score: 0.4247


In [57]:
meta_predictions

array([0.84957804, 0.11631489, 0.17309547, ..., 0.06279431, 0.26104053,
       0.08339355])

In [58]:
count = 4

In [59]:
from datetime import datetime
import pytz
# カウント変数をインクリメント
count += 1
# 日本時間を取得
japan_tz = pytz.timezone('Asia/Tokyo')
now = datetime.now(japan_tz)
timestamp = now.strftime("%Y%m%d_%H%M%S")

file_name = f"/content/drive/MyDrive/Signate/2024summer/lightcat_{timestamp}_{count:03d}.csv"
ss[1] = meta_predictions
ss.to_csv(file_name, header=False, index=False)